# CIFAR-10 图像分类 - 卷积神经网络 (CNN)

该项目使用 Keras 构建和训练卷积神经网络对 CIFAR-10 图像进行分类，内容包括：

- 加载和预处理数据
- 构建优化的 CNN 模型
- 模型训练和测试
- 绘制训练曲线
- 保存和加载模型
- 随机预测并可视化结果
- 分析模型结构与参数对性能的影响

In [ ]:
# 导入必要的库
import time
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import random

In [ ]:
# 加载和预处理数据
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)

In [ ]:
# 构建优化后的 CNN 模型
model = models.Sequential([
    layers.Conv2D(64, (3,3), activation='relu', padding='same', input_shape=(32,32,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer=optimizers.RMSprop(learning_rate=0.0005),
              loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# 模型训练
start_train = time.time()
history = model.fit(x_train, y_train_cat, epochs=15, batch_size=64, validation_split=0.2, verbose=2)
end_train = time.time()

In [ ]:
# 模型测试
start_test = time.time()
test_loss, test_acc = model.evaluate(x_test, y_test_cat, verbose=2)
end_test = time.time()

print(f"训练时间: {end_train - start_train:.2f} 秒")
print(f"测试时间: {end_test - start_test:.2f} 秒")
print(f"测试准确率: {test_acc:.4f}, 测试损失: {test_loss:.4f}")

In [ ]:
# 绘制训练过程曲线
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='训练损失')
plt.plot(history.history['val_loss'], label='验证损失')
plt.title('损失曲线')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='训练准确率')
plt.plot(history.history['val_accuracy'], label='验证准确率')
plt.title('准确率曲线')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 保存模型
model.save('Cifar10.h5')
print("模型已保存为 Cifar10.h5")

In [ ]:
# 加载模型并随机预测10张图像
model_loaded = load_model('Cifar10.h5')
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

plt.figure(figsize=(12,6))
for i in range(10):
    idx = random.randint(0, len(x_test) - 1)
    img = x_test[idx]
    true_label = y_test[idx][0]
    pred = model_loaded.predict(img.reshape(1,32,32,3))
    pred_label = np.argmax(pred)
    plt.subplot(2,5,i+1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"真: {class_names[true_label]}\n预测: {class_names[pred_label]}")
plt.tight_layout()
plt.show()

### 实验结果分析：
- 增加卷积层数和卷积核数量有助于提高准确率。
- 使用 Dropout 可有效缓解过拟合。
- RMSprop 在小学习率下训练较稳定。
- 模型训练时间与结构复杂度成正比，需权衡。